In [39]:
import torchvision

transform = torchvision.transforms.Compose([
    torchvision.transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [43]:
NUM_TIMESTEPS = 128

model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, NUM_TIMESTEPS)

device = torch.device('cuda')
model = model.to(device)

Next, we'll create the optimizer that we'll use to train the neural network.  We'll use the ``Adam`` optimizer with default parameters.

In [44]:
optimizer = torch.optim.Adam(model.parameters())

Finally, execute the following cell to train the neural network for the number of epochs specified

In [35]:
import torch.utils.data
import os
import cv2
import PIL.Image
import torch
import glob


class CircuitDataset(torch.utils.data.Dataset):
    def __init__(self, directory, num_timesteps, transform=None):
        self.directory = directory
        self.image_paths = sorted(glob.glob(os.path.join(self.directory, '*.jpg')))
        self.num_timesteps = num_timesteps
        self.transform = transform
        
    def __len__(self):
        return len(self.image_paths) - self.num_timesteps
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        image = PIL.Image.fromarray(image)
        width = image.width
        height = image.height
        
        if self.transform is not None:
            image = self.transform(image)
            
        target = torch.zeros(self.num_timesteps)
        for i in range(self.num_timesteps):
            path = os.path.splitext(os.path.basename(self.image_paths[idx + i]))[0]
            x = float(int(path.split('_')[1]) - 50) / 50.0
            target[i] = x
        
        return image, target

In [40]:
dataset = CircuitDataset('circuit_log', NUM_TIMESTEPS, transform=transform)

In [41]:
import torch

loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=32,
    shuffle=True
)

In [46]:
import torch.nn.functional as F

EPOCHS = 1

for epoch in range(EPOCHS):
    
    epoch_loss = 0.0
    
    for image, target in iter(loader):
        
        image = image.to(device)
        target = target.to(device)
        
        optimizer.zero_grad()
        
        output = model(image)
        
        loss = F.mse_loss(output, target)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss
        
    epoch_loss /= len(loader)
    
    print('%d: %f' % (epoch, epoch_loss))

0: 0.001613


Finally, we save the model for use in the live demo notebook.

In [47]:
torch.save(model.state_dict(), 'circuit_model.pth')